In [2]:
cd /Users/zoltan/Dropbox/Channels/channelmapper

/Users/zoltan/Dropbox/Channels/channelmapper


In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate
import pandas as pd
import shapefile
from descartes import PolygonPatch
from tqdm.notebook import tqdm, trange
import datetime

import channelmapper as cm

%matplotlib qt

%config InlineBackend.figure_format = 'svg'
plt.rcParams['pdf.fonttype'] = 42

# I. Create bars, scrolls, and BTI polygons for a sequence of channels

### 1. Create initial set of bars

In [30]:
dates = ['19860716', '19870703', '19880806', '19890708', '19900727', '19910612', '19920724', '19930719', 
         '19940807', '19950810', '19960609', '19970714', '19980717', '19990720', '20000706', '20010725',
         '20030715', '20040701', '20050704', '20060723', '20070811', '20080712', '20090731', '20100702',
         '20110806', '20130710', '20140713', '20150801', '20160718', '20170721', '20180622']

In [5]:
# this assumes that the channel banks exist as shapefiles in the directory called 'dirname' and the files are named
# as 'lb_2018' (left bank) and 'rb_2018' (right bank)

ts = len(dates) # this takes ~40 minutes
dirname = '/Users/zoltan/Dropbox/Channels/channelmapper/shapefiles/'
fig = plt.figure()
ax = fig.add_subplot(111)
bars, erosions, chs, all_chs, jumps, cutoffs = cm.create_bars(dates[:ts], 1e6, dirname, ax)
plt.axis('equal');

### 2. Create BTI polygons

In [6]:
# this assumes that the channel centerlines and small channel polygons exist as shapefiles 
# in the directory called 'dirname' and they are named as 'cline_2018' and 'polys_2018' (for an image from 2018)

W = 350.0 # channel width in meters
kl = 60.0 # migration rate constant in meters/year
deltas = 25.0 # distance between points along centerline (meters)

all_polys = []
all_btis = []
all_curvs = []
all_migr_rates = []
all_s = []
for ts in trange(len(dates)-1):
    x1, x2, y1, y2, polys, bti, curv, migr_rate, s = cm.get_bti_polys(dates,dirname,ts1=ts,ts2=ts+1,deltas=deltas,W=W,kl=kl)
    all_polys.append(polys)
    all_btis.append(bti)
    all_curvs.append(curv)
    all_migr_rates.append(migr_rate)
    all_s.append(s)

### 3. Organize everything into bar hierarchy and save BTI polygons

In [7]:
Bars, bti_ages, bti_areas, bti_polys, bti_indices = cm.create_bar_hierarchy(bars, cutoffs, dates, all_polys, all_btis)

In [512]:
import geopandas as gpd

dirname = '/Users/zoltan/Dropbox/Channels/channelmapper/shapefiles/'
btidata = {'age': bti_ages, 'area': bti_areas, 'bti': bti_indices}
df = pd.DataFrame(btidata)
gdf = gpd.GeoDataFrame(df, geometry=bti_polys)
gdf.crs = {'init' :'epsg:32620'}
gdf.to_file(dirname + 'bti_polygons_updated_'+dates[-1][:4]+'.shp')

In [15]:
# plot scroll bars colored by age
fig = cm.plot_scroll_bars(bars, cutoffs, dates)

In [16]:
# plot scroll bars colored by BTI values
fig = cm.plot_btis(Bars, lw = 0.1, vmin = -2, vmax = 2)

# II. Summarize migration rates and channel widths

Indices of segments with valid migration rate values:

0: 1986-1987: 1038, 13607

1: 1987-1988: 26, 9625; 10235, 12652

2: 1988-1989: all is good

3: 1989-1990: 520, 13186

4: 1990-1991: 28, 5050; 5360, -1

5: 1991-1992: 897, 7604; 7900, 16024

6: 1992-1993: 0, 13481

7: 1993-1994: 0, 4739; 4936, -1

8: 1994-1995: 0, 9520; 9822, 14173

9: 1995-1996: 51, 13815

10: 1996-1997: 0, 13560

11: 1997-1998: 0, 3016; 4035, 13640

12: 1998-1999: 623, -1

13: 1999-2000: 26, 13530

14: 2000-2001: 0, 15716

15: 2001-2003: 105, 4723; 4952, 15847

16: 2003-2004: all is good

17: 2004-2005: 35, 14480

18: 2005-2006: all is good

19: 2006-2007: 0, 4354; 4512, 13893

20: 2007-2008: 0, 411; 616, 4472; 4594, -1

21: 2008-2009: all is good

22: 2009-2010: 91, 2384; 3149, 15187

23: 2010-2011: 0, 13018; 13576, 14869

24: 2011-2013: 26, 7787; 7993, 13861

25: 2013-2014: all is good

26: 2014-2015: 0, 14093

27: 2015-2016: all is good

28: 2016-2017: 94, 14487

29: 2017-2018: 373, 3245; 3439, 3761; 4486, -1

In [17]:
plt.figure()
plt.plot(all_migr_rates[29], '.-')

In [18]:
migr_rate_0 = all_migr_rates[0].copy(); 
migr_rate_0[:1038] = np.nan; 
migr_rate_0[13607:] = np.nan

migr_rate_1 = all_migr_rates[1].copy()
migr_rate_1[:26] = np.nan
migr_rate_1[9625:10235] = np.nan
migr_rate_1[12652:] = np.nan

migr_rate_2 = all_migr_rates[2].copy()

migr_rate_3 = all_migr_rates[3].copy()
migr_rate_3[:520] = np.nan 
migr_rate_3[13186:] = np.nan 

migr_rate_4 = all_migr_rates[4].copy()
migr_rate_4[:28] = np.nan
migr_rate_4[5050:5360] = np.nan

migr_rate_5 = all_migr_rates[5].copy()
migr_rate_5[:897] = np.nan
migr_rate_5[7604:7900] = np.nan
migr_rate_5[16024:] = np.nan

migr_rate_6 = all_migr_rates[6].copy()
migr_rate_6[13481:] = np.nan

migr_rate_7 = all_migr_rates[7].copy()
migr_rate_7[4739:4936] = np.nan

migr_rate_8 = all_migr_rates[8].copy()
migr_rate_8[9520:9822] = np.nan
migr_rate_8[14173:] = np.nan

migr_rate_9 = all_migr_rates[9].copy()
migr_rate_9[:51] = np.nan
migr_rate_9[13815:] = np.nan

migr_rate_10 = all_migr_rates[10].copy()
migr_rate_10[13560:] = np.nan

migr_rate_11 = all_migr_rates[11].copy()
migr_rate_11[3016:4035] = np.nan
migr_rate_11[13640:] = np.nan

migr_rate_12 = all_migr_rates[12].copy()
migr_rate_12[:623] = np.nan

migr_rate_13 = all_migr_rates[13].copy()
migr_rate_13[:26] = np.nan
migr_rate_13[13530:] = np.nan

migr_rate_14 = all_migr_rates[14].copy()
migr_rate_14[15716:] = np.nan

migr_rate_15 = all_migr_rates[15].copy()
migr_rate_15[:105] = np.nan
migr_rate_15[4723:4952] = np.nan
migr_rate_15[15847:] = np.nan

migr_rate_16 = all_migr_rates[16].copy()

migr_rate_17 = all_migr_rates[17].copy()
migr_rate_17[:35] = np.nan
migr_rate_17[14480:] = np.nan

migr_rate_18 = all_migr_rates[18].copy()
        
migr_rate_19 = all_migr_rates[19].copy()
migr_rate_19[4354:4512] = np.nan
migr_rate_19[13893:] = np.nan

migr_rate_20 = all_migr_rates[20].copy()
migr_rate_20[411:616] = np.nan
migr_rate_20[4472:4594] = np.nan

migr_rate_21 = all_migr_rates[21].copy()

migr_rate_22 = all_migr_rates[22].copy()
migr_rate_22[:91] = np.nan
migr_rate_22[2384:3149] = np.nan
migr_rate_22[15187:] = np.nan

migr_rate_23 = all_migr_rates[23].copy()
migr_rate_23[13018:13576] = np.nan
migr_rate_23[14869:] = np.nan

migr_rate_24 = all_migr_rates[24].copy()
migr_rate_24[:26] = np.nan
migr_rate_24[7787:7993] = np.nan
migr_rate_24[13861:] = np.nan

migr_rate_25 = all_migr_rates[25].copy()

migr_rate_26 = all_migr_rates[26].copy()
migr_rate_26[14093:] = np.nan

migr_rate_27 = all_migr_rates[27].copy()

migr_rate_28 = all_migr_rates[28].copy()
migr_rate_28[:94] = np.nan
migr_rate_28[14487:] = np.nan

migr_rate_29 = all_migr_rates[29].copy()
migr_rate_29[:373] = np.nan
migr_rate_29[3245:3439] = np.nan
migr_rate_29[3761:4486] = np.nan

In [19]:
all_migr_rates_clean = [] # list for migration rates with no bad values
all_migr_rates_clean.append(migr_rate_0)
all_migr_rates_clean.append(migr_rate_1)
all_migr_rates_clean.append(migr_rate_2)
all_migr_rates_clean.append(migr_rate_3)
all_migr_rates_clean.append(migr_rate_4)
all_migr_rates_clean.append(migr_rate_5)
all_migr_rates_clean.append(migr_rate_6)
all_migr_rates_clean.append(migr_rate_7)
all_migr_rates_clean.append(migr_rate_8)
all_migr_rates_clean.append(migr_rate_9)
all_migr_rates_clean.append(migr_rate_10)
all_migr_rates_clean.append(migr_rate_11)
all_migr_rates_clean.append(migr_rate_12)
all_migr_rates_clean.append(migr_rate_13)
all_migr_rates_clean.append(migr_rate_14)
all_migr_rates_clean.append(migr_rate_15)
all_migr_rates_clean.append(migr_rate_16)
all_migr_rates_clean.append(migr_rate_17)
all_migr_rates_clean.append(migr_rate_18)
all_migr_rates_clean.append(migr_rate_19)
all_migr_rates_clean.append(migr_rate_20)
all_migr_rates_clean.append(migr_rate_21)
all_migr_rates_clean.append(migr_rate_22)
all_migr_rates_clean.append(migr_rate_23)
all_migr_rates_clean.append(migr_rate_24)
all_migr_rates_clean.append(migr_rate_25)
all_migr_rates_clean.append(migr_rate_26)
all_migr_rates_clean.append(migr_rate_27)
all_migr_rates_clean.append(migr_rate_28)
all_migr_rates_clean.append(migr_rate_29)

In [20]:
# list for migration rates with no bad values and no nans (for plotting/statistics):
all_migr_rates_clean_no_nans = []
for m_rates in all_migr_rates_clean:
    all_migr_rates_clean_no_nans.append(np.abs(m_rates[np.isnan(m_rates)==0]))
    
time_intervals = []
for i in range(len(dates)-1):
    time_intervals.append(dates[i][:4] + ' - ' + dates[i+1][:4])

In [21]:
all_migr_rates_clean_no_nans_for_plot = []
for m_rates in all_migr_rates_clean_no_nans:
    all_migr_rates_clean_no_nans_for_plot += list(m_rates)
time_intervals_for_plot = []
count = 0
for time_interval in time_intervals:
    time_intervals_for_plot += [time_interval]*len(all_migr_rates_clean_no_nans[count])
    count += 1

In [22]:
import seaborn as sns
plt.figure()
chart = sns.boxplot(x = time_intervals_for_plot, y = all_migr_rates_clean_no_nans_for_plot,
            flierprops = {'marker':'.', 'markersize':2})
chart.set_xticklabels(chart.get_xticklabels(), rotation=90)
plt.ylim(-10, 600)
plt.ylabel('migration rate (m/year)')
plt.plot([0, 29],[60, 60], 'k--');

In [26]:
widths_new = []
for date in dates:
    filename = '/Users/zoltan/Dropbox/Channels/channelmapper/shapefiles/polys_'+date[:4]
    sf = shapefile.Reader(filename).shapes()
    w = []
    for i in range(len(sf)):
        dist1 = np.linalg.norm(np.array(sf[i].points[0]) - np.array(sf[i].points[1]))
        dist2 = np.linalg.norm(np.array(sf[i].points[2]) - np.array(sf[i].points[3]))
        dist = 0.5*(dist1 + dist2)
        w.append(dist)
    widths_new.append(w)

In [27]:
widths_new_for_plot = []
for w in widths_new:
    widths_new_for_plot += w
dates_for_plot = []
count = 0
for date in dates:
    dates_for_plot += [date[:4]]*len(widths_new[count])
    count += 1

In [28]:
plt.figure()
chart = sns.boxplot(x = dates_for_plot, y = widths_new_for_plot, flierprops = {'marker':'.', 'markersize':2})
chart.set_xticklabels(chart.get_xticklabels(), rotation=90)
plt.ylim(100, 800)
plt.ylabel('channel width (m)')
plt.plot([0, 30],[350, 350], 'k--');